In [142]:
import pandas as pd

### Mapping the original annotation files with review files 

In [163]:
file_mapping = {
    'Sai_Annotated.csv': 'jaskirat_reviewed.csv',
    'jaskirat_annotated.csv': 'Sai_Reviewed.csv',
    'hari_annotated.xlsx': 'jaya_reviewed.xlsx',
    'jaya_annotation.xlsx': 'hari_reviewed.xlsx'
}

In [164]:
def read_file(file_name):
    'function to read a csv or excel file'
    if i[-1] == 'x':
        df = pd.read_excel(file_name)
        if i[:4] == 'hari':
            df.columns = ['Unnamed: 0', 'url', 'title', 'job_description', 'seniority_level',
       'employment_type', 'skills', 'class_name', 'experience']
    else:
        try:
            df = pd.read_csv(file_name)
        except:
            df = pd.read_csv(file_name,encoding="MacRoman")
    df = df[['url', 'title', 'job_description', 'seniority_level',
       'employment_type', 'skills', 'class_name', 'experience']]
    df = df[~df['url'].isna()]
    return df

### Combining all the annotation files into one dataframe and all the review files into another dataframe

In [158]:
annotations = []
reviews = []
for i in file_mapping:
    annotations.append(read_file(i))
    reviews.append(read_file(file_mapping[i]))
df_annot = pd.concat(annotations, ignore_index = True)
df_rev = pd.concat(reviews, ignore_index = True)
df_annot.head()

,url,title,job_description,seniority_level,employment_type,skills,class_name,experience
0,https://www.linkedin.com/jobs/view/full-stack-...,Full Stack Software Engineer,['SpaceX was founded under the belief that a f...,Entry level,Full-time,"python,c#.net,go,scala,java,docker,kubernetes,...",full_stack_engineer,junior
1,https://www.linkedin.com/jobs/view/associate-d...,Intermediate Full-Stack Web Developer (C#/MSSQ...,['Splashdot is looking for a Intermediate Full...,Not Applicable,Full-time,"C#,ASP.NET,SQL,T-SQL,Javascript,jQuery,Bootstr...",full_stack_engineer,intermediate
2,https://www.linkedin.com/jobs/view/associate-d...,Frontend Software Engineer,"[""ResponsibilitiesFounded in 2012, ByteDance's...",Mid-Senior level,Full-time,"HTML,CSS,JavaScript,React,Node,SQL,NoSQL,Kotli...",full_stack_engineer,intermediate
3,https://www.linkedin.com/jobs/view/associate-d...,Full Stack Engineer,['About The JobTome is building across a broad...,Entry level,Full-time,"React,TypeScript,Node,PostgreSQL",full_stack_engineer,junior
4,https://www.linkedin.com/jobs/view/associate-d...,Full Stack Developer (Remote),['MonetizeMore builds industry-leading ad tech...,Mid-Senior level,Full-time,"Node,JavaScript,Python,Flask,PostgreSQL,React,...",full_stack_engineer,intermediate


In [159]:
df_rev.head()

,url,title,job_description,seniority_level,employment_type,skills,class_name,experience
0,https://www.linkedin.com/jobs/view/full-stack-...,Full Stack Software Engineer,['SpaceX was founded under the belief that a f...,Entry level,Full-time,"angular,kubernetes,software,operating system,d...",full_stack_engineer,intermediate
1,https://www.linkedin.com/jobs/view/associate-d...,Intermediate Full-Stack Web Developer (C#/MSSQ...,['Splashdot is looking for a Intermediate Full...,Not Applicable,Full-time,"testing,realm,framework,server,react,design,co...",full_stack_engineer,na
2,https://www.linkedin.com/jobs/view/associate-d...,Frontend Software Engineer,"[""ResponsibilitiesFounded in 2012, ByteDance's...",Mid-Senior level,Full-time,"framework,front end,component,scalability,mobi...",full_stack_engineer,na
3,https://www.linkedin.com/jobs/view/associate-d...,Full Stack Engineer,['About The JobTome is building across a broad...,Entry level,Full-time,"postgresql,react,typescript",full_stack_engineer,intermediate
4,https://www.linkedin.com/jobs/view/associate-d...,Full Stack Developer (Remote),['MonetizeMore builds industry-leading ad tech...,Mid-Senior level,Full-time,"shell,flask,ruby,deployment,software,database,...",full_stack_engineer,intermediate


In [165]:
def skill_set(skill_string):
    'Function to extract a set of skills form a string of skills'
    if skill_string and type(skill_string)==str:
        return set([i.strip().lower() for i in skill_string.split(',')])
    else:
        return set()

In [161]:
df_annot['skills'] = df_annot['skills'].apply(skill_set)
df_rev['skills'] = df_rev['skills'].apply(skill_set)

### Taking the union of skills from two sets of annotations by two different annotators

In [162]:
for i in range(len(df_rev)):
    df_annot.loc[i]['skills'] = df_annot.loc[i]['skills'].union(df_rev.loc[i]['skills'])
df_annot['skills'] = df_annot['skills'].apply(lambda x: ','.join([i.strip()[1:-1] for i in str(x)[1:-1].split(',')]))
df_annot['skills']

0      sql,python,server,react,database,scala,c#.net,...
1      asp net,asp.net,sql,c#,server,realm,html/css,r...
2      ambiguity,sql,server,swift,broadcasting,react,...
3                       postgresql,typescript,react,node
4      git,python,react,deployment,node,linux,ruby,sc...
                             ...                        
273                             mcafee,sophos,crowdstrke
274            cobit,lan,csf,nist,soc,glba,hipaa,pci,wan
275    pki,python,udp,http,linux,ssh,dns,tcp,powershe...
276                   cism,nist,siem,iso,security+,cissp
277                                   iso,cobit,nist,iam
Name: skills, Length: 278, dtype: object

In [166]:
df_annot.to_csv('final_annotated_data.csv', index=False)